In [2]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import numpy as np

# Read in Data

In [3]:
%run "Data Collection and Cleaning.ipynb"

field_goal_percentage
(0.4108217029858827, 5.770790700932277e-05)
--------------------
opp_defensive_rebounds
(-0.4020568981009735, 8.56415288065974e-05)
--------------------
opp_field_goal_percentage
(-0.43353894761032724, 1.9683540812484233e-05)
--------------------
opp_three_point_field_goals
(-0.41061671206255884, 5.825051567272131e-05)
--------------------
opp_two_point_field_goal_percentage
(-0.44225511626329284, 1.2758623050835261e-05)
--------------------
two_point_field_goal_percentage
(0.40956845115249607, 6.110024595107985e-05)
--------------------
W
(1.0, 0.0)
--------------------


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [4]:
#create dataframe with the high correlation columns, and split up into train and test data
all_var_df = df_merged.copy().drop(columns=["name","abbreviation"])

print(all_var_df.columns)

train_data_set = all_var_df[all_var_df["Season"] < 2019].drop(columns=["Season"])
y_train = train_data_set['W'].squeeze()
x_train = train_data_set.drop(columns=['W'])

test_data_set = all_var_df[all_var_df["Season"] == 2019].drop(columns=["Season"])
y_test = test_data_set['W'].squeeze()
x_test = test_data_set.drop(columns=['W'])



Index(['assists', 'blocks', 'defensive_rebounds', 'field_goal_attempts',
       'field_goal_percentage', 'field_goals', 'free_throw_attempts',
       'free_throw_percentage', 'free_throws', 'games_played',
       'minutes_played', 'offensive_rebounds', 'opp_assists', 'opp_blocks',
       'opp_defensive_rebounds', 'opp_field_goal_attempts',
       'opp_field_goal_percentage', 'opp_field_goals',
       'opp_free_throw_attempts', 'opp_free_throw_percentage',
       'opp_free_throws', 'opp_offensive_rebounds', 'opp_personal_fouls',
       'opp_points', 'opp_steals', 'opp_three_point_field_goal_attempts',
       'opp_three_point_field_goal_percentage', 'opp_three_point_field_goals',
       'opp_total_rebounds', 'opp_turnovers',
       'opp_two_point_field_goal_attempts',
       'opp_two_point_field_goal_percentage', 'opp_two_point_field_goals',
       'personal_fouls', 'points', 'rank', 'steals',
       'three_point_field_goal_attempts', 'three_point_field_goal_percentage',
       'three_po

In [5]:
#Train Linear Rregression Model
regr = LinearRegression().fit(x_train, y_train)

y_predict_train = regr.predict(x_train)
y_predict_test = regr.predict(x_test)

Since sklearn accuracy function is based on exact matches, we will test against a window of accuracy. For instance, if we are looking for an exact match, then the window will be 0. If we look for a window of 2, and the actual number of wins is 70, then it will be considered accurate between 68 and 72

In [6]:
def calc_accuracy_rate(match_bool):
    true_count = 0
    total_matches = len(match_bool)
    for match in match_bool:
        if match:
            true_count += 1

    return true_count / total_matches

In [7]:
#Read linear Regression Model's Accuracy
def get_accuracy(num_wins_window, y_predict):
    # if number of wins predicted is within 5 of the actual number of wins 
    # then this will be marked as a "success"
    # (for example if actual number of wins is 65, "correct window" is 60-70)
    # this can easily be adjusted based on the users desired level of accuracy

    round_y_predict = []

    for i,y in enumerate(y_predict):
        wins_diff = abs(y - y_train[i])
        
        if wins_diff <= num_wins_window:
            match = True
        else:
            match = False
        round_y_predict.append(match)

        
    # round_y_predict_arr = np.array(round_y_predict)
    train_accuracy = calc_accuracy_rate(round_y_predict)

    return train_accuracy




In [8]:
train_accuracy_0 = get_accuracy(0,y_predict_train)
print("Exact Accuracy:",train_accuracy_0)

train_accuracy_1 = get_accuracy(1,y_predict_train)
print("Rate of correct wins within 1:", train_accuracy_1)



Exact Accuracy: 0.11666666666666667
Rate of correct wins within 1: 1.0


The rate is at 100% within a window of 1. While exciting, this is not definitive since this is also the set we trained the dataset on. so our next step will be to compare to the test dataset.

In [15]:
test_accuracy_0 = get_accuracy(0,y_predict_test)
print("Exact Accuracy:",test_accuracy_0)

test_accuracy_1 = get_accuracy(1,y_predict_test)
print("Rate of correct wins within 1:", test_accuracy_1)

test_accuracy_3 = get_accuracy(3,y_predict_test)
print("Rate of correct wins within 3:", test_accuracy_3)

test_accuracy_5 = get_accuracy(5,y_predict_test)
print("Rate of correct wins within 5:", test_accuracy_5)

test_accuracy_10 = get_accuracy(10, y_predict_test)
print("Rate of correct wins within 10:", test_accuracy_10)


Exact Accuracy: 0.0
Rate of correct wins within 1: 0.03333333333333333
Rate of correct wins within 3: 0.13333333333333333
Rate of correct wins within 5: 0.16666666666666666
Rate of correct wins within 10: 0.36666666666666664
Rate of correct wins within 20: 0.8333333333333334
